# Tracking Trump: Electoral Votes

**[Morning Consult](https://morningconsult.com)** has a page called **[Tracking Trump](https://morningconsult.com/tracking-trump/)** that summarizes the presidential approval polls on a state-by-state basis, and tells you the number of states in which Trump has a net positive or net negative approval rating. But if you're thinking about the 2020 election, you don't care about the number of *states*, you care about the number of *electoral votes*. Let's do some computation to figure that out.

In [1]:
import urllib.request
import re
import collections 

State = collections.namedtuple('State', 'name ev app dis')

EVs = dict(AL=9,  AK=3,  AZ=11, AR=6,  CA=55, CO=9,  CT=7,  DE=3,  DC=3,  FL=29, 
           GA=16, HI=4,  ID=4,  IL=20, IN=11, IA=6,  KS=6,  KY=8,  LA=8,  ME=4, 
           MD=10, MA=11, MI=16, MN=10, MS=6,  MO=10, MT=3,  NE=5,  NV=6,  NH=4,  
           NJ=14, NM=5,  NY=29, NC=15, ND=3,  OH=18, OK=7,  OR=7,  PA=20, RI=4,  
           SC=9,  SD=3,  TN=11, TX=38, UT=6,  VT=3,  VA=13, WA=12, WV=5,  WI=10, WY=3)

def parse_page(url):
    "Fetch data from the website and parse into a list of `State`s."
    with urllib.request.urlopen(url) as response:
        html = response.read().decode('utf-8')
        rows = re.findall(r'<tr(.*?)</tr>', html, re.S)
        return [parse_row(row) for row in rows[1:]]
    
def parse_row(row):
    "Parse an html string into a `State`."
    # Rows are: |name|delta|Jan17 app|Jan 17 dis|Jan 17 err|now app|now dis|now err|
    name, *_, app, dis, _ = re.findall('>([^>]*?)</td', row.replace('%', ''))
    return State(name, EVs[name], int(app), int(dis))

def net(state): "Net approval minus disapproval."; return state.app - state.dis 
def undecided(state): "Percent of undecided voters"; return 100 - state.app - state.dis
def allocation(amount, Δ): return amount if Δ > 0 else amount/2 if Δ == 0 else 0

states = parse_page('https://morningconsult.com/tracking-trump/')

def EV(swing=0, states=states):
    "Total electoral votes in net positive states (plus 1/2 net zero), after applying swing."
    return sum(allocation(state.ev, net(state) + swing) 
               for state in states)

In [2]:
EV()

126.5

This says that Trump has a net positive approval in states with a total of **126.5** electoral votes. It is a fraction because I allocate half the electoral votes for states with a net zero approval. You need **270** to win. The 126.5 number is down from 164 in Jan 2019, and 448 in Jan 2017.

But of course these are approval polls, not ballots, and don't translate directly to votes. Things can change; the election is a long ways away, we don't know who's running, we don't know if there are third party candidate(s), and we don't know if there is systematic bias in the polling data. In the table below, I list the number of electoral votes Trump would get assuming the vote is determined by the current net approval plus a swing of between 0 and 11 percentage points across the board in every state. We see he would need an **11** percent upswing from the polling data in order to put together wore than 270 electoral votes, and a 6 point swing to exceed the total Clinton got in 2016.

In [3]:
{swing: EV(swing) for swing in range(12)}

{0: 126.5,
 1: 148,
 2: 161.0,
 3: 174,
 4: 196.0,
 5: 218,
 6: 230.0,
 7: 242,
 8: 247.5,
 9: 253,
 10: 269.5,
 11: 288.0}

Now we show each state, sorted by net approval, with their number of votes, net approval, and the three approval percentages: positive, negative, undecided:

In [4]:
for s in sorted(states, key=net):
    print(f'{s.name}: {s.ev:2d} EV, net {net(s):+3d} (+{s.app} -{s.dis} ?{undecided(s)})')

DC:  3 EV, net -65 (+16 -81 ?3)
VT:  3 EV, net -35 (+31 -66 ?3)
MA: 11 EV, net -31 (+33 -64 ?3)
CA: 55 EV, net -30 (+33 -63 ?4)
MD: 10 EV, net -30 (+33 -63 ?4)
HI:  4 EV, net -29 (+34 -63 ?3)
WA: 12 EV, net -26 (+35 -61 ?4)
CT:  7 EV, net -24 (+36 -60 ?4)
NY: 29 EV, net -24 (+36 -60 ?4)
IL: 20 EV, net -23 (+37 -60 ?3)
OR:  7 EV, net -22 (+37 -59 ?4)
NH:  4 EV, net -19 (+39 -58 ?3)
NJ: 14 EV, net -19 (+39 -58 ?3)
RI:  4 EV, net -19 (+39 -58 ?3)
CO:  9 EV, net -18 (+39 -57 ?4)
MN: 10 EV, net -18 (+39 -57 ?4)
NM:  5 EV, net -18 (+39 -57 ?4)
WI: 10 EV, net -16 (+40 -56 ?4)
DE:  3 EV, net -15 (+41 -56 ?3)
MI: 16 EV, net -15 (+40 -55 ?5)
IA:  6 EV, net -14 (+41 -55 ?4)
NV:  6 EV, net -13 (+42 -55 ?3)
ME:  4 EV, net -11 (+43 -54 ?3)
PA: 20 EV, net -10 (+43 -53 ?4)
VA: 13 EV, net -10 (+43 -53 ?4)
AZ: 11 EV, net  -8 (+44 -52 ?4)
OH: 18 EV, net  -6 (+45 -51 ?4)
UT:  6 EV, net  -6 (+45 -51 ?4)
FL: 29 EV, net  -4 (+46 -50 ?4)
NC: 15 EV, net  -4 (+46 -50 ?4)
GA: 16 EV, net  -2 (+47 -49 ?4)
MO: 10 E

Note: Michigan, Wisconsin, and Pennsylvania (which Trump won in 2016) are all double-digit negative now. How is Trump doing in the states that border the proposed wall? Surprisingly poorly: -18 in New Mexico, -8 in Arizona, net zero in Texas, and (not surprisingly), -30 in California.

Below are all the states with more than 5% undecided: the empty set. This is evidence that most people have made up their mind. (Although that could change as a clear challenger emerges from the field.)

In [5]:
{s.name for s in states if undecided(s) > 5}

set()

Here's a table tracking the numbers over time; I'll update each month, but if I forget, you can run this notebook yourself to see the latest numbers.

|Date|Trump EV|Swing needed|5%+ undecided|
|----|--------|------------|------|
|Feb 19 | 126.5 | 11% | 0 |
|Jan 19 | 164   |  7% | 3    |
|Jan 17 | 448   | -10% | 51 |